In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, f1_score, accuracy_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, auc, roc_curve
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
from statsmodels.stats.outliers_influence import variance_inflation_factor  
import warnings
import pickle
import os
import json


warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

In [2]:
cols = ['ts', 'uid', 'id.orig_h', 'id.orig_p',
        'id.resp_h', 'id.resp_p', 'proto', 'service',
        'duration',  'orig_bytes', 'resp_bytes',
        'conn_state', 'local_orig', 'local_resp',
        'missed_bytes',  'history', 'orig_pkts',
        'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes',
        'tunnel_parents', 'label']

In [5]:
out_dir = f'/home/meryem.janati/lustre/nlp_team-um6p-st-sccs-id7fz1zvotk/IDS/janati/IDS/Datasets/unsw/Zeek/17-02-2015/timeout60'
df = pd.read_csv(out_dir+"/UNSW-NB15_zeek_60.csv")
df.head()

,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,orig_bytes,resp_bytes,conn_state,local_orig,local_resp,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,tunnel_parents,id,Attack
0,1.424223e+09,C9dAU041N3HqL5K6kd,59.166.0.7,61953,149.171.126.2,24075,tcp,-,0.05984,236,0,S0,F,F,0,Sa,2,112,2,104,-,0,Benign
1,1.424223e+09,CJ1IZu1BweGN34fIia,149.171.126.5,6881,59.166.0.4,65360,tcp,-,0.122888,64154,68,SF,F,F,0,DTaTdtAFf,68,81674,37,2009,-,1,Benign
2,1.424223e+09,CCur3k3zLi0KpW626,59.166.0.9,24935,149.171.126.2,5190,tcp,-,0.004947,216,814,SF,F,F,0,ShADTdtfFa,12,1064,12,2260,-,2,Benign
3,1.424223e+09,CWTE0433ZFQyGfmTzl,59.166.0.6,37415,149.171.126.1,143,tcp,-,0.137649,68,225,SF,F,F,0,DTadtAfF,8,502,14,1094,-,3,Benign
4,1.424223e+09,CZylbB1X08yVSPfALe,59.166.0.7,25718,149.171.126.2,23652,tcp,-,0.074317,235,22843,SF,F,F,3967,ShdGtAgtfFa,32,1672,35,40046,-,4,Benign


In [4]:
def save_scores(timeout, meanScores, stdScore):
    results = {
        'Timeout': timeout,
        'Mean of all scores': meanScores,
        'Std of all Scores': stdScores

    }

    with open(f'../Checkpoints/RF/RF_unsw_zeek_{timeout}.json', 'w') as f:
        json.dump(results, f, indent=4)


In [5]:
timeouts = ['default', 0.5, 1, 2, 3, 4, 5, 6, 10, 30, 60]

### Training

In [6]:
best_f1 = 0
worst_f1 = 1


best_mean, worst_mean, best_std, worst_std = None, None, None, None
save=True


for timeout in timeouts:
    print("Processing timeout : ", timeout)
    if timeout =='default':
        out_dir1 = f'/home/meryem.janati/lustre/nlp_team-um6p-st-sccs-id7fz1zvotk/IDS/janati/IDS/Datasets/unsw/Zeek/17-02-2015/{timeout}/UNSW-NB15_zeek_{timeout}.csv'
        out_dir2 = f'/home/meryem.janati/lustre/nlp_team-um6p-st-sccs-id7fz1zvotk/IDS/janati/IDS/Datasets/unsw/Zeek/22-01-2015//{timeout}/UNSW-NB15_zeek_{timeout}.csv'
    else:
        out_dir1 = f'/home/meryem.janati/lustre/nlp_team-um6p-st-sccs-id7fz1zvotk/IDS/janati/IDS/Datasets/unsw/Zeek/17-02-2015/timeout{timeout}/UNSW-NB15_zeek_{timeout}.csv'
        out_dir2 = f'/home/meryem.janati/lustre/nlp_team-um6p-st-sccs-id7fz1zvotk/IDS/janati/IDS/Datasets/unsw/Zeek/22-01-2015/timeout{timeout}/UNSW-NB15_zeek_{timeout}.csv'

    df1 = pd.read_csv(out_dir1)
    df2 = pd.read_csv(out_dir2)
    
    df = pd.concat([df1, df2], ignore_index=True, sort=False)
    
    
    df = df.drop(columns=['id', 'uid', 'id.orig_h', 'id.resp_h', 'tunnel_parents']) # tunnel_parents is empty

    # Handle missing values (if any)
    df.replace({'orig_bytes': '-'}, '0', inplace=True)
    df['orig_bytes'] = pd.to_numeric(df['orig_bytes'], errors='coerce')
    df['orig_bytes'] = df['orig_bytes'].fillna(0).astype('int64')

    df.replace({'resp_bytes': '-'}, '0', inplace=True)
    df['resp_bytes'] = pd.to_numeric(df['resp_bytes'], errors='coerce')
    df['resp_bytes'] = df['resp_bytes'].fillna(0).astype('int64')


    df.replace({'duration': '-'}, '0', inplace=True)
    df['duration'] = pd.to_numeric(df['duration'], errors='coerce')
    df['duration'] = df['duration'].fillna(0).astype('float64')

    df['service'] = df['service'].replace('-', np.nan)
    df['history'] = df['history'].replace('-', np.nan)

    # Convert categorical variables to numerical using Label Encoding
    # Encode protocol and service types
    label_encoders = {}
    for column in ['proto', 'service', 'conn_state', 'history', 'local_orig', 'local_resp']:
        if column in df.columns:
            le = LabelEncoder()
            df[column] = le.fit_transform(df[column])
            label_encoders[column] = le

    # Split df into features and labels
    X = df.drop(columns=['Attack'])  # Assuming 'label' is the target variable
    y = df['Attack']
    
    accuracy, f1, precision, recall =[], [], [], []
    skf= StratifiedKFold(n_splits=5,random_state=None)
    skf.get_n_splits(X,y)
    
    for (train_index, test_index), i in zip(skf.split(X, y), range(5)):
        X_train,X_test=X.iloc[train_index],X.iloc[test_index]
        y_train,y_test=y.iloc[train_index],y.iloc[test_index]


        le = LabelEncoder()
        y_train = le.fit_transform(y_train)
        
        
        # Map unseen values to '<unknown>'
        y_test = y_test.map(lambda s: '<unknown>' if s not in le.classes_ else s)

        # Add '<unknown>' to classes and transform
        le.classes_ = np.append(le.classes_, '<unknown>')
        y_test = le.transform(y_test)
        
        
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

        # Initialize and train Extra Trees Classifier
        clf = RandomForestClassifier(random_state=42)
        clf.fit(X_train, y_train)
        pred = clf.predict(X_test)
        
        f1Score = f1_score(y_true=y_test, y_pred=pred, average='macro')
        accScore=accuracy_score(y_test, pred)
        precScore = precision_score(y_test, pred, average='macro')
        recScrore = recall_score(y_test, pred, average='macro')
                             
        f1.append(f1Score)
        accuracy.append(accScore)
        precision.append(precScore)
        recall.append(recScrore)
        print('Fold: ', i, 'done!')

    meanScores, stdScores = {}, {}
    
    meanScores['f1Mean'] = np.array(f1).mean()
    meanScores['accMean'] = np.array(accuracy).mean()
    meanScores['recMean'] = np.array(recall).mean()
    meanScores['precMean'] = np.array(precision).mean()
    
    stdScores['f1Std'] = np.array(f1).std()
    stdScores['accStd'] = np.array(accuracy).std()
    stdScores['recStd'] = np.array(recall).std()
    stdScores['precStd'] = np.array(precision).std()
    
    print("Mean of all scores: ", meanScores)
    print("Std of all scores: ", stdScores)


    if save:
        save_scores(timeout, meanScores, stdScores)

    if meanScores['f1Mean'] > best_f1: 
        best_timeout = timeout
        best_mean = meanScores
        best_std = stdScores
        best_f1 = meanScores['f1Mean']
    
    if meanScores['f1Mean'] <= worst_f1: 
        
        worst_timeout = timeout
        worst_mean = meanScores
        worst_std = stdScores
        worst_f1 = meanScores['f1Mean']
               
    print('_______________________________________________')

Processing timeout :  default
Fold:  0 done!
Fold:  1 done!
Fold:  2 done!
Fold:  3 done!
Fold:  4 done!
Mean of all scores:  {'f1Mean': 0.45709872311529576, 'accMean': 0.9721313506514331, 'recMean': 0.46088931841555025, 'precMean': 0.563145976732896}
Std of all scores:  {'f1Std': 0.09458164082617261, 'accStd': 0.007988089252538525, 'recStd': 0.13359655805536133, 'precStd': 0.08698361908514503}
_______________________________________________
Processing timeout :  0.5
Fold:  0 done!
Fold:  1 done!
Fold:  2 done!
Fold:  3 done!
Fold:  4 done!
Mean of all scores:  {'f1Mean': 0.44089404167461616, 'accMean': 0.9723172546054464, 'recMean': 0.45139189543512404, 'precMean': 0.5363973338866368}
Std of all scores:  {'f1Std': 0.10123122879850557, 'accStd': 0.007854635512958638, 'recStd': 0.12760667806131457, 'precStd': 0.09976752676545028}
_______________________________________________
Processing timeout :  1
Fold:  0 done!
Fold:  1 done!
Fold:  2 done!
Fold:  3 done!
Fold:  4 done!
Mean of all 

In [9]:
print("Best Timeout Combination: ", best_timeout)
print("Mean Scores (Best): ", best_mean)
print('Std Scores (Best):', best_std)

Best Timeout Combination:  default
Mean Scores (Best):  {'f1Mean': 0.45709872311529576, 'accMean': 0.9721313506514331, 'recMean': 0.46088931841555025, 'precMean': 0.563145976732896}
Std Scores (Best): {'f1Std': 0.09458164082617261, 'accStd': 0.007988089252538525, 'recStd': 0.13359655805536133, 'precStd': 0.08698361908514503}


In [10]:
print("worst Timeout Combination: ", worst_timeout)
print("Mean Scores (Worst): ", worst_mean)
print('Std Scores (Worst):', worst_std)

worst Timeout Combination:  0.5
Mean Scores (Worst):  {'f1Mean': 0.44089404167461616, 'accMean': 0.9723172546054464, 'recMean': 0.45139189543512404, 'precMean': 0.5363973338866368}
Std Scores (Worst): {'f1Std': 0.10123122879850557, 'accStd': 0.007854635512958638, 'recStd': 0.12760667806131457, 'precStd': 0.09976752676545028}


In [11]:

results = {
    'Best score': {
        'Best Timeout': best_timeout,
        'Mean Scores (Best)': best_mean,
        'Std Scores (Best)': best_std,
    },
    
    'Worst score': {
        'Worst Timeout': worst_timeout,
        'Mean Scores (Worst)': worst_mean,
        'Std Scores (Worst)': worst_std,
    },
    
    'Difference': {
        'Accuracy': (best_mean['accMean'] - worst_mean['accMean'])*100,
        'F1 Score': (best_mean['f1Mean'] - worst_mean['f1Mean'])*100,
        'Precision': (best_mean['precMean'] - worst_mean['precMean'])*100,
        'Recall': (best_mean['recMean'] - worst_mean['recMean'])*100
    }
}



with open('../results/RF_unsw_zeek.json', 'w') as f:
    json.dump(results, f, indent=4)